## Semantic Kernel: Ramp-Up based on SK's Documentation

To get the latest version of SK Python package, use:

``` bash
pip install --upgrade semantic-kernel
```

## 📒 Notebook 3: Filters

### 🪜 Step 1: Configure environment

In [1]:
# Import required packages
import asyncio
import logging
import os
from typing import Any, Callable, Coroutine

from semantic_kernel import Kernel
from semantic_kernel.functions import kernel_function
from semantic_kernel.connectors.ai.open_ai import AzureChatCompletion, OpenAIChatPromptExecutionSettings
from semantic_kernel.contents import ChatHistory, ChatMessageContent, TextContent
from semantic_kernel.contents.utils.author_role import AuthorRole
from semantic_kernel.exceptions import OperationCancelledException
from semantic_kernel.filters import FilterTypes, FunctionInvocationContext
from semantic_kernel.functions import FunctionResult

In [2]:
# Set Azure OpenAI backend variables
AOAI_DEPLOYMENT = os.getenv("AZURE_OPENAI_API_DEPLOY")
AOAI_ENDPOINT = os.getenv("AZURE_OPENAI_API_BASE")
AOAI_API_VERSION = os.getenv("AZURE_OPENAI_API_VERSION")

### 🪜 Step 2: Add AOAI Chat Completion service

In [3]:
# Initialise kernel
kernel = Kernel()

In [4]:
# Configure logging
logging.basicConfig(level=logging.ERROR)
logger = logging.getLogger(__name__)

In [5]:
# Add Azure OpenAI chat completion
chat_completion = AzureChatCompletion(
    deployment_name = AOAI_DEPLOYMENT,
    endpoint = AOAI_ENDPOINT,
    api_version = AOAI_API_VERSION,
    service_id = "azure_openai_chat",
)

kernel.add_service(chat_completion)

### 🪜 Step 3: Define content blocking filter

In [6]:
# Define custom filter to block input with forbidden content
async def content_blocking_filter(
    context: FunctionInvocationContext,
    next: Callable[[FunctionInvocationContext], Coroutine[Any, Any, None]],
) -> None:
    """
    A filter that checks for forbidden words in the user's input.
    If a forbidden word is found, it blocks the function invocation
    and sets a custom result.
    """

    FORBIDDEN_WORD = "badword"
    user_input = context.arguments.get("input", "")
    # chat_history is passed as an argument to the invoked function,
    # so it will be available in context.arguments
    chat_history_from_args = context.arguments.get("chat_history")

    if FORBIDDEN_WORD in user_input.lower():
        print(f"- Filter Action: Detected forbidden word '{FORBIDDEN_WORD}'. Blocking function invocation. ---\n")
        blocked_message = ChatMessageContent(
            role=AuthorRole.ASSISTANT,
            items=[TextContent(text="I cannot process requests containing prohibited words. Please try again.")],
            metadata={"filtered": True}
        )
        context.result = FunctionResult(
            function=context.function.metadata,
            value=blocked_message
        )
        if chat_history_from_args:
            chat_history_from_args.add_message(message=blocked_message)
        return
    
    print(f"- Filter Action: No forbidden word found. Proceeding with function invocation. ---\n")
    await next(context)

In [7]:
# Add custom filter to the kernel for function invocation events
kernel.add_filter(FilterTypes.FUNCTION_INVOCATION, content_blocking_filter)

### 🪜 Step 4: Define a Simple Chat Plugin

In [8]:
# Define basic chat plugin
class ChatPlugin:
    @kernel_function(name="Chat", description="Responds to a chat message.")
    async def chat(self, chat_history: ChatHistory, input: str) -> ChatMessageContent:
        """
        Simple chat function that uses the kernel's chat completion service.
        """
        chat_history.add_user_message(input)

        chat_service = kernel.get_service("azure_openai_chat")

        response = await chat_service.get_chat_message_contents(
            chat_history = chat_history,
            settings = OpenAIChatPromptExecutionSettings(
                max_tokens = 150,
                temperature = 0.7
            )
        )
        
        # Return the first response
        if response and len(response) > 0:
            return response[0]
        else:
            return ChatMessageContent(
                role = AuthorRole.ASSISTANT,
                items = [TextContent(text="I'm sorry, I couldn't generate a response.")]
            )

In [9]:
# Add plugin to the kernel
kernel.add_plugin(ChatPlugin(), plugin_name="ChatPlugin")

KernelPlugin(name='ChatPlugin', description=None, functions={'Chat': KernelFunctionFromMethod(metadata=KernelFunctionMetadata(name='Chat', plugin_name='ChatPlugin', description='Responds to a chat message.', parameters=[KernelParameterMetadata(name='chat_history', description=None, default_value=None, type_='ChatHistory', is_required=True, type_object=<class 'semantic_kernel.contents.chat_history.ChatHistory'>, schema_data={'type': 'object', 'properties': {'messages': {'type': 'array', 'items': {'type': 'object', 'properties': {'inner_content': {'type': ['object', 'null'], 'properties': {}}, 'ai_model_id': {'type': ['string', 'null']}, 'metadata': {'type': 'object', 'additionalProperties': {'type': 'object', 'properties': {}}}, 'content_type': {'type': 'object'}, 'tag': {'type': 'object'}, 'role': {'type': 'string', 'enum': ['system', 'user', 'assistant', 'tool', 'developer']}, 'name': {'type': ['string', 'null']}, 'items': {'type': 'array', 'items': {'anyOf': [{'type': 'object', 'prop

### 🪜 Step 5: Demo filter in action

In [10]:
# Define sample user inputs 
user_inputs = [
    {"user_input": "Tell me a story with a badword in it."},
    {"user_input": "Tell me a story about a brave knight."},
    {"user_input": "I like to talk about good things, not badword."},
    {"user_input": "What is the capital of France?"},
]

In [11]:
# Helper function to test scenarios
async def test_scenarios():
    chat_history = ChatHistory()
    for user_input in user_inputs:
        user_message = user_input["user_input"]
        print(f"\n-------------------------------")
        print(f"- User Input: {user_message} ---\n")
        
        # Initialise chat history
        history = ChatHistory()

        # Invoke the chat function
        try:
            response = await kernel.invoke(
                plugin_name = "ChatPlugin",
                function_name = "Chat",
                chat_history = history,
                input = user_message,
            )
        
            # Check if the response was from the filter or the LLM
            if hasattr(response, 'value') and hasattr(response.value, 'metadata'):
                if response.value.metadata and response.value.metadata.get("filtered"):
                    print(f"- Assistant Response (from Filter): {response.value.content}")
                else:
                    print(f"- Assistant Response (from LLM): {response.value.content}")
            else:
                print(f"Assistant Response: {response}")

        except OperationCancelledException as e:
            print(f"Operation cancelled by filter: {e}")
        except Exception as e:
            print(f"An unexpected error occurred: {e}")

In [12]:
# Run all test scenarios
await test_scenarios()


-------------------------------
- User Input: Tell me a story with a badword in it. ---

- Filter Action: Detected forbidden word 'badword'. Blocking function invocation. ---

- Assistant Response (from Filter): I cannot process requests containing prohibited words. Please try again.

-------------------------------
- User Input: Tell me a story about a brave knight. ---

- Filter Action: No forbidden word found. Proceeding with function invocation. ---

- Assistant Response (from LLM): Once upon a time, in a kingdom nestled between towering mountains and lush forests, there lived a brave knight named Sir Cedric. Known throughout the land for his courage and kindness, Sir Cedric was beloved by the people and respected by the king.

One day, a terrible dragon descended upon the kingdom, breathing fire and casting fear into the hearts of all. The dragon demanded a treasure, or it would burn the villages to ash. The king, desperate to save his people, called upon Sir Cedric to face the b